## Homework 4, Problem 4 Classification on real data

ECE C143A/C243A, Spring Quarter 2022, Prof. J.C. Kao, TAs T. Monsoor, W. Yu

## Background
Neural prosthetic systems can be built based on classifying neural activity related to planning. As described in class, this is analogous to mapping patterns of neural activity to keys on a keyboard.
In this problem, we will apply the results of Problems 1 and 2 to real neural data. The neural data were recorded using a 100-electrode array in premotor cortex of a macaque monkey1. The dataset can be found on CCLE as `ps4_realdata.mat`.

The following describes the data format. The `.mat` file is loaded into Python as a dictionary with two keys: `train_trial` contains the training data and `test_trial` contains the test data. Each of these contains spike trains recorded simultaneously from 97 neurons while the monkey reached 91 times along each of 8 different reaching angles.

The spike train recorded from the $i_{th}$ neuron on the $n_{th}$ trial of the $k_{th}$ reaching angle is accessed as 

`data['train_trial'][n,k][1][i,:]`

where n = 0,...,90, k = 0,...,7, and i = 0, . . . , 96.  The [1] in between [n,k] and [i,:] does not mean anything for this assignment and is simply an "artifact" of how the data is structured. A spike train is represented as a sequence of zeros and ones, where time is discretized in 1 ms steps. A zero indicates that the neuron did not spike in the 1 ms bin, whereas a one indicates that the neuron spiked once in the 1 ms bin. The structure test trial has the same format as train trial.

Each spike train is 700 ms long (and thus represented by an array of length 700).  This comprises a 200ms baseline period (before the reach target turned on), a 500ms planning period (after the reach target turned on).  Because it takes time for information about the reach target to arrive in premotor cortex (due to the time required for action potentials to propagate and for visual processing), we will ignore the first 150ms of the planning period.  *** FOR THIS PROBLEM, we will take spike counts for each neuron within a single 200ms bin starting 150ms after the reach target turns on. ***

In other words, to calculate firing rates, you will calculate it over the 200ms window: 

`data['train_trial'][n,k][1][i,350:550]`

In [85]:
import numpy as np
import numpy.matlib as npm
import matplotlib.pyplot as plt
import scipy.special
import scipy.io as sio
import math

data = sio.loadmat('ps4_realdata.mat') # load the .mat file.
NumTrainData = data['train_trial'].shape[0]
NumClass = data['train_trial'].shape[1]
NumTestData = data['test_trial'].shape[0]

# Reloading any code written in external .py files.
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


### (a) (8 points) 
Fit the ML parameters of model i) to the training data (91 × 8 observations of a length 97 array of neuron firing rates). 

To calculate the firing rates, use a single 200ms bin starting from 150ms after the target turns on.  This corresponds to using `data['train_trial'][n,k][1][i, 350:550]` to calculate all firing rates.  This corresponds to a 200ms window that turns on 150ms after the reach turns on.

Then, use these parameters to classify the test data (91 × 8 data points) according to the decision rule (1). What is the percent of test data points correctly classified?

In [90]:
##4a

# Calculate the firing rates.

trainDataArr =  np.zeros((NumClass,NumTrainData,97)) # contains the firing rates for all neurons on all 8 x 91 trials in the training set
testDataArr =  np.zeros((NumClass,NumTestData,97)) # for the testing set.

for classIX in range(NumClass):
    for trainDataIX in range(NumTrainData):   
        trainDataArr[classIX,trainDataIX,:] = np.sum(data['train_trial'][trainDataIX,classIX][1][:,350:550],1)
    for testDataIX in range(NumTestData):        
        testDataArr[classIX,testDataIX,:]=np.sum(data['test_trial'][testDataIX,classIX][1][:,350:550],1)
#====================================================#
# YOUR CODE HERE:
#   Fit the ML parameters of model i) to training data
#====================================================#
n_rates = []
for classk in range(8):
    mov_arr = []
    for neuron in range(97):
        neuron_arr = []
        for datas in range(91):
            neuron_arr.append(trainDataArr[classk, datas, neuron] * 5) #multiply by 5 to get firing rate
        mov_arr.append(neuron_arr)
    n_rates.append(mov_arr)

means = []
for mov in n_rates:
    mov_arr = []
    for neuron in mov:
        avg = np.mean(neuron)
        mov_arr.append(avg)
    means.append(mov_arr)
np_mean_arr = np.empty([8,97])
for mov in range(8):
    for neuron in range(97):
        np_mean_arr[mov][neuron] = means[mov][neuron]
conv = ([vars for vars in n_rates[0]])
class_0_cov = np.cov(conv[0:])
conv = ([vars for vars in n_rates[1]])
class_1_cov = np.cov(conv[0:])
conv = ([vars for vars in n_rates[2]])
class_2_cov = np.cov(conv[0:])
conv = ([vars for vars in n_rates[3]])
class_3_cov = np.cov(conv[0:])
conv = ([vars for vars in n_rates[4]])
class_4_cov = np.cov(conv[0:])
conv = ([vars for vars in n_rates[5]])
class_5_cov = np.cov(conv[0:])
conv = ([vars for vars in n_rates[6]])
class_6_cov = np.cov(conv[0:])
conv = ([vars for vars in n_rates[7]])
class_7_cov = np.cov(conv[0:])

sum_covs = (class_0_cov + class_1_cov + class_2_cov + class_3_cov + class_4_cov 
            + class_5_cov + class_6_cov + class_7_cov)

######MODEL 1######
modParam1 = {}
modParam1['pi'] = np.array((1/8, 1/8, 1/8, 1/8, 1/8, 1/8, 1/8, 1/8,))
modParam1['mean'] = np_mean_arr
modParam1['cov'] = sum_covs/8


#######Test Model 1##########
right = 0
guesses = 0

for classk in range(8):
    for datas in range(91):
        Z = (trainDataArr[classk, datas, ] ) * 5 #multiply by 5 to get firing rate
        prob_arr = []
        inv_cov = np.linalg.inv(modParam1['cov'])
        
        for probs in range(8):
            diff_mean = Z - modParam1['mean'][probs]
            diff_mean_t = np.transpose(diff_mean)
            prob_class =  -.5 *np.matmul(np.matmul(diff_mean_t, inv_cov), diff_mean) 
            prob_arr.append(prob_class)
        max_probs = np.max(prob_arr)
        guess = prob_arr.index(max_probs)
        if (guess == classk):
            right+=1
        guesses+=1
print(f'Accuracy is {right/guesses}')
#====================================================#
# END YOUR CODE
#====================================================# 


#====================================================#
# YOUR CODE HERE:
#   Classify the test data and print the accuracy
#====================================================#

#====================================================#
# END YOUR CODE
#====================================================# 


Accuracy is 0.9807692307692307


#### Question:
What is the percent of test data points correctly classified?

#### Your answer: 
98% were correctly classified

### (b) (6 points) 
Repeat part (a) for model ii). You `should encounter a Python error` when classifying the test data. What is this error? Why did the Python error occur? What would we need to do to correct this error?

To be concrete, the output of this cell should be a `Python error` and that's all fine.  But we want you to understand what the error is so we can fix it later.


In [87]:
##4b

#====================================================#
# YOUR CODE HERE:
# Fit the ML parameters of model ii) to training data
#====================================================#
######MODEL 2######
modParam2 = {}
modParam2['pi'] = np.array((1/8, 1/8, 1/8, 1/8, 1/8, 1/8, 1/8, 1/8,))
modParam2['mean'] = np_mean_arr
modParam2['cov'] = [class_0_cov, class_1_cov, class_2_cov, class_3_cov, class_4_cov, class_5_cov, class_6_cov, class_7_cov]

#######Test Model 2##########
right = 0
guesses = 0
#print(modParam2['cov'])
for classk in range(8):
    for datas in range(91):
        Z = (trainDataArr[classk, datas, ] ) * 5 #multiply by 5 to get firing rate
        prob_arr = []
        
        for probs in range(8):
            inv_cov = np.linalg.inv(modParam2['cov'][probs])
            diff_mean = Z - modParam2['mean'][probs]
            diff_mean_t = np.transpose(diff_mean)
            log_det_cov = np.log(np.linalg.det(modParam2['cov'][probs]))
            prob_class = -.5 * np.matmul(np.matmul(diff_mean_t, inv_cov), diff_mean) - (.5 *log_det_cov)
            prob_arr.append(prob_class)
        max_probs = np.max(prob_arr)
        guess = prob_arr.index(max_probs)
        if (guess == classk):
            right+=1
        guesses+=1
print(f'Accuracy is {right/guesses}')


#====================================================#
# END YOUR CODE
#====================================================# 


LinAlgError: Singular matrix

#### Question:
Why did the python error occur? What would we need to do to correct this error?

#### Your answer:  
Part of the Covariance matrix is not the correct size due to neurons being silent over all samples for a class. 

### (c) (8 points) 
Correct the problem from part (b) by detecting and then removing offending neurons that cause the error. Now, what is the percent of test data points correctly classified? Is it higher or lower than your answer to part (a)? Why might this be?

In [88]:
##4c
neuronsToRemove = []
#====================================================#
# YOUR CODE HERE:
#   Detect and then remove the offending neurons, so that 
#   you no longer run into the bug in part (b).
#====================================================#
for mov in means:
    index = 0
    for neuron in mov:   
        if (np.mean(neuron)== 0):
            neuronsToRemove.append(index)
        index+=1
#====================================================#
# END YOUR CODE
#====================================================# 
##
#====================================================#
# YOUR CODE HERE:
# Fit the ML parameters,classify the test data and print the accuracy
#====================================================#

def remove_mul_ele(input_list,list_of_indexes):   
    list_back = input_list
    removed = 0
    for index in list_of_indexes:
        list_back.pop(index-removed)
        removed+=1
    return list_back
neuronsToRemove = sorted(list(dict.fromkeys(neuronsToRemove)))
num_removed = len(neuronsToRemove)
tot_neurons = 97 - num_removed

new_rates = []
for mov in n_rates:
    new_mov = remove_mul_ele(mov, neuronsToRemove)
    new_rates.append(new_mov)

    
means = []
for mov in new_rates:
    mov_arr = []
    for neuron in mov:
        avg = np.mean(neuron)
        mov_arr.append(avg)
    means.append(mov_arr)
np_mean_arr = np.empty([8,97-num_removed])
for mov in range(8):
    for neuron in range(97-num_removed):
        np_mean_arr[mov][neuron] = means[mov][neuron]
conv = ([vars for vars in new_rates[0]])
class_0_cov = np.cov(conv[0:])
conv = ([vars for vars in new_rates[1]])
class_1_cov = np.cov(conv[0:])
conv = ([vars for vars in new_rates[2]])
class_2_cov = np.cov(conv[0:])
conv = ([vars for vars in new_rates[3]])
class_3_cov = np.cov(conv[0:])
conv = ([vars for vars in new_rates[4]])
class_4_cov = np.cov(conv[0:])
conv = ([vars for vars in new_rates[5]])
class_5_cov = np.cov(conv[0:])
conv = ([vars for vars in new_rates[6]])
class_6_cov = np.cov(conv[0:])
conv = ([vars for vars in new_rates[7]])
class_7_cov = np.cov(conv[0:])

modParam2 = {}
modParam2['pi'] = np.array((1/8, 1/8, 1/8, 1/8, 1/8, 1/8, 1/8, 1/8,))
modParam2['mean'] = np_mean_arr
modParam2['cov'] = [class_0_cov, class_1_cov, class_2_cov, class_3_cov, class_4_cov, class_5_cov, class_6_cov, class_7_cov]


#######Test Model 1##########
right = 0
guesses = 0
for classk in range(8):
    for datas in range(91):
        Z = (trainDataArr[classk, datas, ] ) * 5 #multiply by 5 to get firing rate
        inds_to_delete = sorted(neuronsToRemove, reverse=True) # [5,3,1]
        for inds in inds_to_delete:
            Z = np.delete(Z, inds) 
        prob_arr= []
        for probs in range(8):
            inv_cov = np.linalg.inv(modParam2['cov'][probs])
            diff_mean = Z - modParam2['mean'][probs]
            diff_mean_t = np.transpose(diff_mean)
            log_det_cov = np.log(np.linalg.det(modParam2['cov'][probs]))
            prob_class = -.5 * np.matmul(np.matmul(diff_mean_t, inv_cov), diff_mean) - (.5 *log_det_cov)
            prob_arr.append(prob_class)
        #print(prob_arr)
        #print(classk)
        max_probs = np.max(prob_arr)
        guess = prob_arr.index(max_probs)
        if (guess == classk):
            right+=1
        guesses+=1
print(f'Accuracy is {right/guesses}')





#====================================================#
# END YOUR CODE
#====================================================# 

Accuracy is 1.0


#### Question:
What is the percent of test data points correctly classified? Is it higher or lower than your answer to part (a)? Why might this be?



#### Your answer:  
It is higher at 100% becuase it uses individual covariance matrices which is more accurate than averaging over all the classes for one cov matrix.

### (d) (8 points) 
Now we classify using a naive Bayes model. Repeat part (a) for model iii). Keep the convention in part (c), where offending neurons were removed from the anal- ysis.

In [84]:
##4d
#====================================================#
# YOUR CODE HERE:
# Fit the ML parameters,classify the test data and print the accuracy
#====================================================#
modParam3 = {}
modParam3['pi'] = np.array((1/8, 1/8, 1/8, 1/8, 1/8, 1/8, 1/8, 1/8,))
modParam3['mean'] = np_mean_arr


right = 0
guesses = 0

##Test
for classk in range(8):
    for datas in range(91):
        Z = (trainDataArr[classk, datas, ]) * 5 #multiply by 5 to get firing rate
        inds_to_delete = sorted(neuronsToRemove, reverse=True) # [5,3,1]
        for inds in inds_to_delete:
            Z = np.delete(Z, inds) 
        
        prob_arr= []
        for probs in range(8):
            prob = 0
            for neuron in range(97 - num_removed):
                lamda = modParam3['mean'][probs][neuron]
                summed = Z[neuron] * np.log(lamda) - lamda
                prob += summed
            prob_arr.append(prob)
            
        max_probs = np.max(prob_arr)
        guess = prob_arr.index(max_probs)
        if (guess == classk):
            right+=1
        guesses+=1
print(f'Accuracy is {right/guesses}')
        
#====================================================#
# END YOUR CODE
#====================================================# 

Accuracy is 0.9313186813186813


#### Question:
what is the percent of test data points correctly classified? 

#### Your answer: 
93%